### Try COLAB

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip3 install torch matplotlib torchmetrics scikit-image segmentation-models-pytorch

# Import

In [1]:
import torch
from torch import nn as nn
from torch.nn import functional as F

import torchmetrics

import wandb

import seaborn as sns

### CUDA

In [2]:
# GPU Support?
if torch.cuda.is_available():
    print("Using the GPU")
    device = torch.device('cuda')
else:
    print("Using the CPU")
    device = torch.device('cpu')

Using the GPU


# Generate Dataset

In [194]:
import random

# Generate a dataset of sequences containing characters 'a', 'b', and 'c' in order
N = 1000 # Size of dataset (number of sequences)
dataset = []

for n in range(N):
    if n % 4 == 0: # 25% of the time add actual member of formal language family
        length = random.randint(1, 6)
        sequence = 'a' * length + 'b' * length + 'c' * length
        dataset.append(sequence)
    else:
        length = random.randint(3, 20)  # Random sequence length between 3 and 20
        counts = [0, 0, 0]

        # Distribute the length among a, b, and c
        for i in range(length):
            counts[random.randint(0, 2)] += 1

        # Ensure alphabetical order and create the sequence
        sequence = 'a' * counts[0] + 'b' * counts[1] + 'c' * counts[2]
        dataset.append(sequence)

In [195]:
# print max length of dataset
max_len = max([len(s) for s in dataset])
max_len # Should be twenty

20